------------

# Spatial Panel Models with Fixed Effects

* **This notebook uses the class `Panel_FE_Lag` and `Panel_FE_Error` from `pysal.spreg`.**

In [1]:
%load_ext watermark
%watermark

Last updated: 2021-01-02T21:14:34.646733-05:00

Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.19.0

Compiler    : Clang 11.0.0 
OS          : Darwin
Release     : 20.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [2]:
import numpy
import libpysal
import spreg

%watermark -w
%watermark -iv

Watermark: 2.1.0

numpy   : 1.19.4
spreg   : 1.2.0.post1
libpysal: 4.3.0



**Open data on NCOVR US County Homicides (3085 areas).**
* First, extract the HR (homicide rates) data in the 70's, 80's and 90's as the dependent variable.
* Data can also be passed in the long format instead of wide format.
  * i.e. a vector with $n \times t$ rows and a single column for the dependent variable, and
  * a matrix of dimension $n \times (t \ast k)$ for the independent variables
* Then, extract RD and PS as independent variables in the regression.

In [3]:
# Open data on NCOVR US County Homicides (3085 areas).
nat = libpysal.examples.load_example("NCOVR")
db = libpysal.io.open(nat.get_path("NAT.dbf"), "r")

# Create spatial weight matrix
nat_shp = libpysal.examples.get_path("NAT.shp")
w = libpysal.weights.Queen.from_shapefile(nat_shp)
w.transform = 'r'

# Define dependent variable
name_y = ["HR70", "HR80", "HR90"]
y = numpy.array([db.by_col(name) for name in name_y]).T

# Define independent variables
name_x = ["RD70", "RD80", "RD90", "PS70", "PS80", "PS90"]
x = numpy.array([db.by_col(name) for name in name_x]).T

--------------------

## Spatial Lag model

Let's estimate a spatial lag panel model with fixed effects:

$$
y = \rho Wy + X\beta + \mu_i + e
$$

In [4]:
fe_lag = spreg.Panel_FE_Lag(
    y, x, w, name_y=name_y, name_x=name_x, name_ds="NAT"
)

In [5]:
print(fe_lag.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG PANEL - FIXED EFFECTS
-----------------------------------------------------------------------
Data set            :         NAT
Weights matrix      :     unknown
Dependent Variable  :          HR                Number of Observations:        9255
Mean dependent var  :      0.0000                Number of Variables   :           3
S.D. dependent var  :      3.9228                Degrees of Freedom    :        9252
Pseudo R-squared    :      0.0319
Spatial Pseudo R-squared:  0.0079
Sigma-square ML     :      14.935                Log likelihood        :  -67936.533
S.E of regression   :       3.865                Akaike info criterion :  135879.066
                                                 Schwarz criterion     :  135900.465

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------

In [6]:
numpy.around(fe_lag.betas, decimals=4)

array([[ 0.8006],
       [-2.6004],
       [ 0.1903]])

------------------------

## Spatial Error model

Now, let's estimate a spatial error panel model with fixed effects:

$$
y = X\beta + \mu_i + v
$$

where

$$
v = \lambda W v + e
$$

In [7]:
fe_error = spreg.Panel_FE_Error(
    y, x, w, name_y=name_y, name_x=name_x, name_ds="NAT"
)

In [8]:
print(fe_error.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL ERROR PANEL - FIXED EFFECTS
-------------------------------------------------------------------------
Data set            :         NAT
Weights matrix      :     unknown
Dependent Variable  :          HR                Number of Observations:        9255
Mean dependent var  :      0.0000                Number of Variables   :           2
S.D. dependent var  :      3.9228                Degrees of Freedom    :        9253
Pseudo R-squared    :      0.0084
Sigma-square ML     :      14.923                Log likelihood        :  -67934.005
S.E of regression   :       3.863                Akaike info criterion :  135872.010
                                                 Schwarz criterion     :  135886.276

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-----------------------------------------------

In [9]:
numpy.around(fe_error.betas, decimals=4)

array([[ 0.8698],
       [-2.9661],
       [ 0.1943]])

------------------